In [1]:
import pandas as pd
import pyarrow as pa

In [2]:
# function to flatten widget list and create name and amount column
def explode_widgets(dataframe):
    data = []
    for index, row in dataframe.iterrows():
        # if widget list is empty, append line with empty widget name and amount
        if len(row["widget_list"]) == 0:
            data.append(
                [row['id'], row['email_anon'], row['age_group'], row['user_score'], row['sub_group_rank'], row['revenue'], None,
                 None, row['location'], row['created_at']])
        else:
            # flatten list, append new line containing the same data as original line, add new columns
            for x in row["widget_list"]:
                wid_name = x['name']
                wid_amount = x['amount']
                data.append([row['id'], row['email_anon'], row['age_group'], row['user_score'], row['sub_group_rank'], row['revenue'],
                             wid_name, wid_amount, row['location'], row['created_at']])
    # create new dataframe from list data
    newdf = pd.DataFrame(data, columns=['id', 'email_anon', 'age_group', 'user_score', 'sub_group_rank', 'revenue', 'widget_name',
                                        'widget_amount', 'location', 'created_at'])
    return newdf


# function to create new table with inverted index
def create_inverted_table(dataframe, columnkey, columnval):
    new_table = {}
    for index, row in dataframe.iterrows():
        # append id to existing key
        if row[columnkey] in new_table:
            new_table[row[columnkey]].append(row[columnval])
        # create new key with id value
        else:
            new_table[row[columnkey]] = [row[columnval]]
    # create new table inverted index
    new_table = pd.DataFrame.from_dict(new_table, orient='index').transpose()
    return new_table

def obtain_original_email(email_dict, email_key):
    email_dict = dict((reversed(x) for x in email_dict.items()))
    return email_dict[email_key]



In [8]:
# create dataframe from json file
df = pd.read_json("data.json", lines="True")
# remove duplicated id and created_at
# duplicated = df[df.duplicated(['id', 'created_at'])]
# print(duplicated)
df = df.drop_duplicates(subset=['id', 'created_at'])
# group by age group, rank user score in new column sub_group_rank
df["sub_group_rank"] = df.groupby("age_group")["user_score"].rank(method='dense')
# create anonymized email list by assigning each unique email to a number
email_dictionary = {emails: i for i, emails in enumerate(df["email"].unique())}
#print(email_dictionary)
df["email_anon"] = df["email"].apply(lambda x: email_dictionary[x])
# original email can be retrieved through the dictionary email_dictionary with obtain_original_email
#print(obtain_original_email(email_dictionary, 2))

In [38]:
df_new = df.explode("widget_list")
df_new["widget_list"] = df_new["widget_list"].fillna("")
df_new["widget_name"] = df_new["widget_list"].apply(lambda x: x['name'] if x!="" else "")
df_new["widget_amount"] = df_new["widget_list"].apply(lambda x: x['amount'] if x!="" else -1)
df_new.drop(["widget_list"], axis=1, inplace=True)
df_new

id                       email  \
0    25abf8a8-9d11-4e55-a003-246e458e9fb3     fguinness0@yolasite.com   
0    25abf8a8-9d11-4e55-a003-246e458e9fb3     fguinness0@yolasite.com   
1    6bf052c9-1307-4b26-a24d-9c47bf747c87           fdebenham1@si.edu   
1    6bf052c9-1307-4b26-a24d-9c47bf747c87           fdebenham1@si.edu   
1    6bf052c9-1307-4b26-a24d-9c47bf747c87           fdebenham1@si.edu   
..                                    ...                         ...   
990  dec2ec8c-d2ef-4e53-82c4-2416e416bbd4  dsjostromri@technorati.com   
991  03040023-4f4b-47e5-80bd-c75d1f4c3ffd     cdawburyrj@buzzfeed.com   
992  a63d3f32-be32-4827-927e-52da0dd31df1           flukockrk@usa.gov   
992  a63d3f32-be32-4827-927e-52da0dd31df1           flukockrk@usa.gov   
993  91cf2719-b39a-4098-9fdb-66f1d2eb0d6a         ggolsworthyrl@is.gd   

     age_group  user_score  revenue       location                created_at  \
0            2    0.467348   351.08      Indonesia 2020-01-31 14:50:26+00:00   
0            2    0.467348   351.08      Indonesia 2020-01-31 14:50:26+00:00   
1            1    0.929755   801.00         Greece 2020-07-16 18:32:48+00:00   
1            1    0.929755   801.00         Greece 2020-07-16 18:32:48+00:00   
1            1    0.929755   801.00         Greece 2020-07-16 18:32:48+00:00   
..         ...         ...      ...            ...                       ...   
990          1    0.509603   156.77      Indonesia 2020-02-07 23:24:48+00:00   
991          2    0.928337   828.55          China 2020-06-23 04:14:28+00:00   
992          1    0.204904   626.76         Russia 2020-02-16 12:21:20+00:00   
992          1    0.204904   626.76         Russia 2020-02-16 12:21:20+00:00   
993          3    0.696549   256.71  United States 2020-07-02 10:04:37+00:00   

     sub_group_rank  email_anon          widget_name  widget_amount  
0             116.0           0   Eumetopias jubatus          70520  
0             116.0           0   Ramphastos tucanus          35573  
1             236.0           1  Bassariscus astutus          44123  
1             236.0           1     Lutra canadensis          52666  
1             236.0           1         Oryx gazella           6977  
..              ...         ...                  ...            ...  
990           145.0         990         Ceryle rudis           9890  
991           231.0         991                                  -1  
992            68.0         992      Redunca redunca          37715  
992            68.0         992   Macropus giganteus          26079  
993           156.0         993    Buteo jamaicensis          63503  

[1781 rows x 11 columns]

Afghanistan  \
0    7f91bf87-b2d2-43c1-9f45-79114a1e58e9   
1                                    None   
2                                    None   
3                                    None   
4                                    None   
..                                    ...   
169                                  None   
170                                  None   
171                                  None   
172                                  None   
173                                  None   

                            Aland Islands  \
0    78b070c2-44cc-440c-a533-0c2db94fd2e4   
1                                    None   
2                                    None   
3                                    None   
4                                    None   
..                                    ...   
169                                  None   
170                                  None   
171                                  None   
172                                  None   
173                                  None   

                                  Albania  \
0    52750cb3-cc86-4e09-a580-b3f080cb7c8e   
1    904553fc-f799-4e43-a796-e04e76840cab   
2    fef784c0-dde6-4e48-b2bb-8f0d4dd62770   
3    a1301998-d2d3-4827-84f3-6d32491fff5e   
4    3e8842fd-bcdc-46c2-bbdf-e9b81373a1f8   
..                                    ...   
169                                  None   
170                                  None   
171                                  None   
172                                  None   
173                                  None   

                           American Samoa  \
0    9f481f67-9d54-4648-b1c4-9dec6c52ee34   
1                                    None   
2                                    None   
3                                    None   
4                                    None   
..                                    ...   
169                                  None   
170                                  None   
171                                  None   
172                                  None   
173                                  None   

                                   Angola  \
0    aa46c42b-7073-478e-a11e-262f821c3f1b   
1                                    None   
2                                    None   
3                                    None   
4                                    None   
..                                    ...   
169                                  None   
170                                  None   
171                                  None   
172                                  None   
173                                  None   

                                Argentina  \
0    c9126756-289e-41f9-846d-90387283b99a   
1    a79cffc6-6cc5-442e-a59f-e039f9aa9936   
2    792a99e8-3249-4c90-bfe3-9f34a355fd5b   
3    bdc6b6d2-c583-4a1c-ba68-3bfa12e880ef   
4    99b65432-1ccf-4ae4-ba5e-7eb1a1c7678e   
..                                    ...   
169                                  None   
170                                  None   
171                                  None   
172                                  None   
173                                  None   

                                  Armenia  \
0    e4c7b0ea-466e-4ada-afef-2f1d475be165   
1    2b1cc159-8fc5-43d7-8cb2-f3f10011ac02   
2    fe320d20-5657-47f9-929f-89ed3798f7eb   
3    5a98d56e-432c-475e-b0c6-08cf555280c4   
4    b7df83b1-c11a-4383-b849-14076a2aeb23   
..                                    ...   
169                                  None   
170                                  None   
171                                  None   
172                                  None   
173                                  None   

                               Azerbaijan  \
0    823aa6ad-de6f-4eea-8640-0d6f4ddfc25f   
1    eb21f364-2906-4c5d-a776-a5e43d902a4d   
2    ea52a827-b348-4cec-9de4-c28d02f4e9a8   
3    16a05619-b7dc-40d6-8297-8f07fbc2b6e8   
4                 

In [33]:
df_new

id                    email  age_group  \
0  25abf8a8-9d11-4e55-a003-246e458e9fb3  fguinness0@yolasite.com          2   
0  25abf8a8-9d11-4e55-a003-246e458e9fb3  fguinness0@yolasite.com          2   
1  6bf052c9-1307-4b26-a24d-9c47bf747c87        fdebenham1@si.edu          1   
1  6bf052c9-1307-4b26-a24d-9c47bf747c87        fdebenham1@si.edu          1   
1  6bf052c9-1307-4b26-a24d-9c47bf747c87        fdebenham1@si.edu          1   
2  c3dc4870-463e-49d3-aba7-383cace03cd5      lleband2@hao123.com          3   
3  42c18d61-34bb-4b27-a2cf-f43728c690f7  sbartolommeo3@imgur.com          4   
3  42c18d61-34bb-4b27-a2cf-f43728c690f7  sbartolommeo3@imgur.com          4   
3  42c18d61-34bb-4b27-a2cf-f43728c690f7  sbartolommeo3@imgur.com          4   
4  e4c7b0ea-466e-4ada-afef-2f1d475be165        ebraben4@xing.com          3   

   user_score  revenue                                        widget_list  \
0    0.467348   351.08    {'name': 'Eumetopias jubatus', 'amount': 70520}   
0    0.467348   351.08    {'name': 'Ramphastos tucanus', 'amount': 35573}   
1    0.929755   801.00   {'name': 'Bassariscus astutus', 'amount': 44123}   
1    0.929755   801.00      {'name': 'Lutra canadensis', 'amount': 52666}   
1    0.929755   801.00           {'name': 'Oryx gazella', 'amount': 6977}   
2    0.473031   637.77                                                      
3    0.315440   588.28         {'name': 'Manouria emys', 'amount': 72401}   
3    0.315440   588.28  {'name': 'Sagittarius serpentarius', 'amount':...   
3    0.315440   588.28  {'name': 'Mirounga angustirostris', 'amount': ...   
4    0.009215   191.75      {'name': 'Mazama americana', 'amount': 35949}   

    location                created_at  sub_group_rank  email_anon  \
0  Indonesia 2020-01-31 14:50:26+00:00           116.0           0   
0  Indonesia 2020-01-31 14:50:26+00:00           116.0           0   
1     Greece 2020-07-16 18:32:48+00:00           236.0           1   
1     Greece 2020-07-16 18:32:48+00:00           236.0           1   
1     Greece 2020-07-16 18:32:48+00:00           236.0           1   
2      China 2019-09-26 05:26:24+00:00           116.0           2   
3    Finland 2020-01-17 09:26:34+00:00            81.0           3   
3    Finland 2020-01-17 09:26:34+00:00            81.0           3   
3    Finland 2020-01-17 09:26:34+00:00            81.0           3   
4    Armenia 2020-01-29 21:55:14+00:00             4.0           4   

                widget_name  widget_amount  
0        Eumetopias jubatus          70520  
0        Ramphastos tucanus          35573  
1       Bassariscus astutus          44123  
1          Lutra canadensis          52666  
1              Oryx gazella           6977  
2                                       -1  
3             Manouria emys          72401  
3  Sagittarius serpentarius          28076  
3   Mirounga angustirostris          83195  
4          Mazama americana          35949

In [ ]:
# flatten widget list and create new columns name and amount
df = explode_widgets(df)
df

id  email_anon  age_group  user_score  \
0     25abf8a8-9d11-4e55-a003-246e458e9fb3           0          2    0.467348   
1     25abf8a8-9d11-4e55-a003-246e458e9fb3           0          2    0.467348   
2     6bf052c9-1307-4b26-a24d-9c47bf747c87           1          1    0.929755   
3     6bf052c9-1307-4b26-a24d-9c47bf747c87           1          1    0.929755   
4     6bf052c9-1307-4b26-a24d-9c47bf747c87           1          1    0.929755   
...                                    ...         ...        ...         ...   
1776  dec2ec8c-d2ef-4e53-82c4-2416e416bbd4         990          1    0.509603   
1777  03040023-4f4b-47e5-80bd-c75d1f4c3ffd         991          2    0.928337   
1778  a63d3f32-be32-4827-927e-52da0dd31df1         992          1    0.204904   
1779  a63d3f32-be32-4827-927e-52da0dd31df1         992          1    0.204904   
1780  91cf2719-b39a-4098-9fdb-66f1d2eb0d6a         993          3    0.696549   

      sub_group_rank  revenue          widget_name  widget_amount  \
0              116.0   351.08   Eumetopias jubatus        70520.0   
1              116.0   351.08   Ramphastos tucanus        35573.0   
2              236.0   801.00  Bassariscus astutus        44123.0   
3              236.0   801.00     Lutra canadensis        52666.0   
4              236.0   801.00         Oryx gazella         6977.0   
...              ...      ...                  ...            ...   
1776           145.0   156.77         Ceryle rudis         9890.0   
1777           231.0   828.55                 None            NaN   
1778            68.0   626.76      Redunca redunca        37715.0   
1779            68.0   626.76   Macropus giganteus        26079.0   
1780           156.0   256.71    Buteo jamaicensis        63503.0   

           location                created_at  
0         Indonesia 2020-01-31 14:50:26+00:00  
1         Indonesia 2020-01-31 14:50:26+00:00  
2            Greece 2020-07-16 18:32:48+00:00  
3            Greece 2020-07-16 18:32:48+00:00  
4            Greece 2020-07-16 18:32:48+00:00  
...             ...                       ...  
1776      Indonesia 2020-02-07 23:24:48+00:00  
1777          China 2020-06-23 04:14:28+00:00  
1778         Russia 2020-02-16 12:21:20+00:00  
1779         Russia 2020-02-16 12:21:20+00:00  
1780  United States 2020-07-02 10:04:37+00:00  

[1781 rows x 10 columns]

In [11]:
# create inverted table for which ids exist in each location
country = create_inverted_table(df, 'location', 'id')
country

Indonesia  \
0    25abf8a8-9d11-4e55-a003-246e458e9fb3   
1    25abf8a8-9d11-4e55-a003-246e458e9fb3   
2    e26f3624-9851-49dd-aaeb-6131078364b9   
3    8119f944-3db3-4896-a9c9-8226dff521c4   
4    8119f944-3db3-4896-a9c9-8226dff521c4   
..                                    ...   
316                                  None   
317                                  None   
318                                  None   
319                                  None   
320                                  None   

                                   Greece  \
0    6bf052c9-1307-4b26-a24d-9c47bf747c87   
1    6bf052c9-1307-4b26-a24d-9c47bf747c87   
2    6bf052c9-1307-4b26-a24d-9c47bf747c87   
3    f0c66a9c-6cca-43be-8891-55964292b3cf   
4    435fe51b-03fa-43d8-a340-6d7557000537   
..                                    ...   
316                                  None   
317                                  None   
318                                  None   
319                                  None   
320                                  None   

                                    China  \
0    c3dc4870-463e-49d3-aba7-383cace03cd5   
1    68478da6-921e-404e-a3c9-3d4510f1769b   
2    68478da6-921e-404e-a3c9-3d4510f1769b   
3    56aea724-77bd-4d6a-b2d9-c267c624f745   
4    92f04197-21ed-4574-8d7f-8b8609a8eb24   
..                                    ...   
316  69ced210-ece5-452d-87f1-6e1da841e9a2   
317  9d77428d-7c25-47cc-978d-d33c73eafeec   
318  9d77428d-7c25-47cc-978d-d33c73eafeec   
319  9d77428d-7c25-47cc-978d-d33c73eafeec   
320  03040023-4f4b-47e5-80bd-c75d1f4c3ffd   

                                  Finland  \
0    42c18d61-34bb-4b27-a2cf-f43728c690f7   
1    42c18d61-34bb-4b27-a2cf-f43728c690f7   
2    42c18d61-34bb-4b27-a2cf-f43728c690f7   
3    b55f9796-4ad5-4ce0-8b08-d16f8d89730b   
4    b55f9796-4ad5-4ce0-8b08-d16f8d89730b   
..                                    ...   
316                                  None   
317                                  None   
318                                  None   
319                                  None   
320                                  None   

                                  Armenia  \
0    e4c7b0ea-466e-4ada-afef-2f1d475be165   
1    e4c7b0ea-466e-4ada-afef-2f1d475be165   
2    2b1cc159-8fc5-43d7-8cb2-f3f10011ac02   
3    2b1cc159-8fc5-43d7-8cb2-f3f10011ac02   
4    2b1cc159-8fc5-43d7-8cb2-f3f10011ac02   
..                                    ...   
316                                  None   
317                                  None   
318                                  None   
319                                  None   
320                                  None   

                                   Canada  \
0    cb5d1348-d9f4-4267-8636-baaf29ca96ae   
1    cb5d1348-d9f4-4267-8636-baaf29ca96ae   
2    cb5d1348-d9f4-4267-8636-baaf29ca96ae   
3    8fddfe26-dc67-4401-82d1-9546e83f1a76   
4    8fddfe26-dc67-4401-82d1-9546e83f1a76   
..                                    ...   
316                                  None   
317                                  None   
318                                  None   
319                                  None   
320                                  None   

                            United States  \
0    cc72a6e0-9d8e-4539-a20c-fef8859deee6   
1    cc72a6e0-9d8e-4539-a20c-fef8859deee6   
2    cc72a6e0-9d8e-4539-a20c-fef8859deee6   
3    15f7617d-40af-4b08-b144-16a9dcea8134   
4    62a261d0-3871-4754-8f7b-751da03b9ce3   
..                                    ...   
316                                  None   
317                                  None   
318                                  None   
319                                  None   
320                                  None   

                                   Zambia  \
0    793c795d-be00-42cc-9c0c-56df19f11f53   
1    7034d6a7-5088-4f78-8673-59ba21d43adf   
2    7034d6a7-5088-4f78-8673-59ba21d43adf   
3    7034d6a7-5088-4f78-8673-59ba21d43adf   
4    6904d851-2428-4

In [ ]:
country[]

#test csv
#country.to_csv('output.csv')
#df.to_csv('ce.csv')

#write cleaned data to parquet file
processed_table = pa.Table.from_pandas(df)
#pq.write_table(processed_table, 'processed_data.parquet')

#write inverted index to parquet file
country_table = pa.Table.from_pandas(country)
#pq.write_table(country_table, 'inverted_country_index.parquet')